# Práctica 3. Reconocimiento de Entidades Nombradas. NER

Andrés González Flores

Procesamiento de Lenguaje Natural

Facultad de Ingeniería, UNAM

## Objetivo

Realizar un reconocimiento de Entidades Nombradas (NER) a partir de un modelo secuencial y con el corpus ‘ner dataset.csv’ que se proporciona.

## Instrucciones

Se deberán seguir los siguientes pasos:

1. Obtener las sentencias a partir del csv. En este archivo, se indica cada inicio de sentencia con ‘Sentence: n’. Se cuenta con 1000 sentencias que conformarán el corpus de entrenamiento y evaluación.
2. Preprocesar los datos.
3. Separar los datos en corpus de entrenamiento (70%) y corpus de evaluación (30%).
4. Entrenar un modelo secuencial a partir del corpus de entrenamiento. Deberán definirse los hiperparámetros.
5. Evaluar el desempeño del sistema a partir del corpus de evaluación y con la métrica de Exactitud (Accuracy).
6. Ejemplificar el reconocimiento de entidades nombradas con 5 sentencias del corpus de evaluación.

## Dessarrollo

In [1]:
import csv
import numpy as np
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from itertools import chain
from tqdm.notebook import tqdm as nbtqdm
from pprint import pprint

In [2]:
# Definición de constantes
SEED = 42
CORPUS_PATH = './ner_dataset.csv'
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<unk>'

np.random.seed(SEED)

### Paso 1. Obtener sentencias.


In [3]:
with open(CORPUS_PATH, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    csvlist = list(reader)

In [4]:
sentences = []
sent_i = []
for row in csvlist[1:]:
    if row[0] is not '':
        sentences.append(sent_i)
        sent_i = []
    sent_i.append((row[1], row[2]))
# Mi algoritmo está feo: agrega la primer sentencia vacía y no agrega la última
del sentences[0]
sentences.append(sent_i)

In [5]:
print(f'Número de oraciones = {len(sentences)}\n')
print('Ejemplos:')
print('\n'.join([' '.join([w for w, tag in sent]) for sent in sentences[:5] ]))

Número de oraciones = 1000

Ejemplos:
Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "
They marched from the Houses of Parliament to a rally in Hyde Park .
Police put the number of marchers at 10000 while organizers claimed it was 1,00,000 .
The protest comes on the eve of the annual conference of Britain 's ruling Labor Party in the southern English seaside resort of Brighton .


### Paso 2. Preprocesamiento

In [6]:
freq_tokens = Counter([w for w,_ in chain(*sentences)])
freq_tokens.most_common(10)

[('the', 1111),
 ('.', 995),
 (',', 637),
 ('in', 576),
 ('of', 568),
 ('to', 505),
 ('a', 449),
 ('and', 391),
 ('The', 241),
 ("'s", 202)]

Agrego el diccionario de entrada

In [7]:
word_to_index = { 
    x[0] : i for i, x in enumerate(freq_tokens.most_common())
    if freq_tokens[x[0]] > 1
}

pprint(list(word_to_index.items())[:10])

[('the', 0),
 ('.', 1),
 (',', 2),
 ('in', 3),
 ('of', 4),
 ('to', 5),
 ('a', 6),
 ('and', 7),
 ('The', 8),
 ("'s", 9)]


Agrego los identificadores UNK, BOS Y EOS al vocabulario de entrada

In [8]:
word_to_index[UNK] = max(word_to_index.values())+1
word_to_index[BOS] = word_to_index[UNK]+1
word_to_index[EOS] = word_to_index[BOS]+1

pprint(list(word_to_index.items())[-10:])

[('collapse', 2102),
 ('Barno', 2103),
 ('reunification', 2104),
 ('Berlin', 2105),
 ('Kohl', 2106),
 ('proud', 2107),
 ('wall', 2108),
 ('<unk>', 2109),
 ('<BOS>', 2110),
 ('<EOS>', 2111)]


Agrego el vocabulario de salida (las etiquetas)

In [9]:
freq_tags = Counter([tag for _,tag in chain(*sentences)])
freq_tags.most_common()

[('', 18843),
 ('B-ge', 583),
 ('B-gpe', 543),
 ('B-rg', 413),
 ('I-per', 400),
 ('B-tim', 358),
 ('B-per', 327),
 ('I-rg', 290),
 ('I-ge', 105),
 ('I-tim', 77),
 ('B-art', 39),
 ('I-gpe', 26),
 ('I-art', 22),
 ('B-eve', 20),
 ('I-eve', 16),
 ('B-nat', 9),
 ('I-nat', 5)]

In [10]:
tag_to_index = {
    y[0] : i for i, y in enumerate(freq_tags.most_common())
}
pprint(tag_to_index)

{'': 0,
 'B-art': 10,
 'B-eve': 13,
 'B-ge': 1,
 'B-gpe': 2,
 'B-nat': 15,
 'B-per': 6,
 'B-rg': 3,
 'B-tim': 5,
 'I-art': 12,
 'I-eve': 14,
 'I-ge': 8,
 'I-gpe': 11,
 'I-nat': 16,
 'I-per': 4,
 'I-rg': 7,
 'I-tim': 9}


Creo los vocabularios inversos

In [11]:
index_to_word = [w for w in word_to_index.keys()]
pprint(index_to_word[::350])
print()

index_to_tag = [tag for tag in tag_to_index.keys()]
pprint(index_to_word)

ers',
 'only',
 'make',
 'bomber',
 'build',
 'ties',
 'national',
 'making',
 'small',
 'house',
 'arrest',
 'much',
 'Organization',
 'negotiations',
 'ABAC',
 'Australian',
 'across',
 'discuss',
 'decision',
 'connection',
 'left',
 'French',
 'well',
 'launched',
 'At',
 'toll',
 '50',
 'person',
 'seven',
 'plans',
 'sides',
 'office',
 'show',
 'site',
 'rate',
 'began',
 'Merck',
 'ministry',
 'south',
 'Malaysia',
 'Japan',
 'development',
 'member',
 'human',
 'Egypt',
 'Western',
 'time',
 'shows',
 'issued',
 'Earlier',
 'key',
 'Venezuela',
 'civilians',
 'Italian',
 'terrorism',
 'billion',
 'campaign',
 'natural',
 'Soviet',
 'economic',
 'Meanwhile',
 'Jihad',
 'Ukraine',
 'Iowa',
 'protesters',
 'carried',
 'One',
 'Labor',
 'Party',
 'ahead',
 'cities',
 'European',
 'Tehran',
 'kidnapped',
 'raid',
 'An',
 'better',
 'used',
 'occurred',
 'largest',
 'soldier',
 'attempt',
 'cut',
 'public',
 'bill',
 'force',
 'close',
 'drug',
 'commander',
 'senior',
 '2004',
 'pr

Creo un método que devuelve siempre un indice del vocabulario o el identificador UNK

In [12]:
def word_safe_vocab_index(w, vocab, ix_UNK):
    try:
        return vocab[w]
    except KeyError:
        return ix_UNK

Armo el corpus de entrenamiento

In [13]:
ix_UNK = word_to_index[UNK]
ix_BOS = word_to_index[BOS]
ix_EOS = word_to_index[EOS]

BOS_append = [(ix_BOS, tag_to_index[''])] # BOS y su etiqueta
EOS_append = [(ix_EOS, tag_to_index[''])] # EOS y su etiqueta

corpus = [
    BOS_append + [ # Agrego BOS y su etiqueta
        # Tupla con el valor del índice de la palabra y el índice de la etiqueta
        (
            word_safe_vocab_index(w, word_to_index, ix_UNK),
            tag_to_index[tag]
        ) # Fin tupla
        for w, tag in sent # Para cada palabra en la sentencia
    ] + EOS_append # Fin append
    for sent in sentences # Por cada sentencia en la lista de sentencias
]
# pprint(corpus[:5])
print('\n\n'.join([' '.join([f'({ix_w}, {ix_tag})' for ix_w, ix_tag in sent]) for sent in corpus[:5] ]))

(2110, 0) (904, 0) (4, 0) (905, 0) (14, 0) (906, 0) (243, 0) (244, 1) (5, 0) (298, 0) (0, 0) (127, 0) (3, 0) (53, 1) (7, 0) (1282, 0) (0, 0) (679, 0) (4, 0) (99, 2) (77, 0) (17, 0) (16, 0) (71, 0) (1, 0) (2111, 0)

(2110, 0) (2109, 0) (4, 0) (177, 0) (31, 0) (3, 0) (0, 0) (680, 0) (551, 0) (0, 0) (458, 0) (59, 0) (459, 0) (2109, 0) (18, 0) (245, 0) (1283, 0) (25, 0) (28, 0) (200, 6) (2109, 0) (460, 0) (2109, 0) (28, 0) (7, 0) (28, 0) (2109, 0) (0, 0) (2109, 0) (1, 0) (28, 0) (2111, 0)

(2110, 0) (134, 0) (906, 0) (17, 0) (0, 0) (2109, 0) (4, 0) (2109, 0) (5, 0) (6, 0) (1284, 0) (3, 0) (2109, 1) (2109, 8) (1, 0) (2111, 0)

(2110, 0) (268, 0) (340, 0) (0, 0) (341, 0) (4, 0) (1285, 0) (22, 0) (1286, 0) (135, 0) (1287, 0) (299, 0) (32, 0) (19, 0) (2109, 0) (1, 0) (2111, 0)

(2110, 0) (8, 0) (298, 0) (552, 0) (15, 0) (0, 0) (1288, 0) (4, 0) (0, 0) (387, 0) (553, 0) (4, 0) (554, 1) (9, 0) (907, 0) (461, 3) (462, 7) (3, 0) (0, 0) (128, 0) (2109, 2) (2109, 0) (2109, 0) (4, 0) (2109, 1) (1, 0) 

### Paso 3. Separar corpus de entrenamiento y evaluacion

In [14]:
train_corpus, eval_corpus = train_test_split(corpus, test_size=0.3, random_state=SEED)
print("Ejemlos del corpus de entrenamiento")
for sent in train_corpus[:5]:
    print(sent)
print()
print("Ejemlos del corpus de evaluacion")
for sent in eval_corpus[:5]:
    print(sent)

Ejemlos del corpus de entrenamiento
[(2110, 0), (123, 0), (13, 0), (0, 0), (692, 0), (341, 0), (4, 0), (178, 0), (1599, 0), (636, 0), (1211, 0), (11, 0), (326, 0), (480, 0), (5, 0), (2109, 0), (129, 0), (2, 0), (171, 0), (1670, 0), (1, 0), (2111, 0)]
[(2110, 0), (8, 0), (487, 0), (19, 0), (1776, 0), (27, 0), (0, 0), (131, 0), (668, 0), (3, 0), (2109, 1), (2, 0), (7, 0), (0, 0), (2109, 0), (4, 0), (50, 6), (667, 4), (2, 0), (6, 0), (2109, 0), (4, 0), (2109, 2), (49, 6), (2109, 4), (2109, 4), (2, 0), (19, 0), (13, 0), (5, 0), (565, 0), (1777, 0), (5, 0), (340, 0), (46, 0), (1778, 0), (15, 0), (2109, 2), (1, 0), (2111, 0)]
[(2110, 0), (443, 0), (51, 0), (56, 0), (2, 0), (606, 2), (144, 6), (93, 4), (2109, 4), (2109, 4), (543, 0), (24, 0), (254, 3), (2109, 0), (11, 0), (331, 0), (538, 0), (82, 0), (4, 0), (224, 2), (9, 0), (175, 0), (2, 0), (348, 1), (1, 0), (2111, 0)]
[(2110, 0), (1507, 0), (609, 0), (39, 0), (2109, 0), (199, 0), (46, 0), (633, 0), (7, 0), (52, 0), (1467, 0), (5, 0), (210

### Paso 4. Selección de modelo y entrenamiento

Usaré un modelo de red neuronal recurrente con una capa de embedding, una capa oculta de recurrencia, y una capa de salida.

La capa de embedding se define como:

$$ x = Cs $$

$C$ es una matriz de dimensión $d\times N_{in}$ y $s$ es la representación one hot de la palabra en la secuencia.

La capa oculta recurrente:

$$ h^{(t)} = \tanh{(Vh^{(t-1)}+Ux^{(t)}+b)} $$

$V$ es una matriz de dimensión $m\times m$, $U$ una matriz de dimensión $d\times m $ y $b$ es el vector de bias, de dimensión $m$.

Y la capa de salida:

$$ y^{(t)} = Softmax(Wh^{(t-1)}+c) $$

En donde $W$ es una matriz de dimensión $N_{out} \times m$

Los valores de $d$ y $m$ son hiperparámetros


In [15]:
class Modelo(object):
    def __init__(self, d, m, N_in, N_out):
        self.d = d # DImensión de capa de embedding
        self.m = m # Dimensión de capa oculta
        self.N_in = N_in # Dimensión de entrada
        self.N_out = N_out # Dimensión de salida

    def inicializar_pesos(self):
        # Capa de embedding
        self.C = np.random.randn(self.d, self.N_in) / np.sqrt(self.N_in)
                
        # Capa recurrente oculta
        self.V = np.random.randn(self.m, self.m) / np.sqrt(self.m)
        self.U = np.random.randn(self.m, self.d) / np.sqrt(self.d)
        self.b = np.zeros(self.m)

        # Capa de salida
        self.W = np.random.randn(self.N_out, self.m) / np.sqrt(self.m) 
        self.c = np.zeros(self.N_out)
        
        # Los mejores pesos del modelo
        self.best_C = self.C        
        self.best_V = self.W
        self.best_U = self.U
        self.best_b = self.b        
        self.best_W = self.W
        self.best_c = self.c
    
    def forward(self, secuencia):
        """Paso hacia adelante de la red. 
        Recibe como argumento una secuencia de entrada de n elementos, estos elementos
        deben ser indices del vocabulario de entrada.
        Da como salida $n$ vectores $h$, y $n$ vectores $y$
        """
        # Tamaño de la secuencia de entrada
        T = len(secuencia)

        # Prealojamiento de salidas por estado de capas oculta y de salida.
        h = np.zeros((T+1, self.m)) # Uso un estado adicional para t = -1
        y = np.zeros((T, self.N_out))

        # Se inicia el vector h[-1] con ceros
        h[-1] = np.zeros(self.m)

        # Propagación hacia adelante de la secuenca de entrada
        for t in range(T):
            # Un paso en la capa embedding se reduce a C[:, s]
            x_t = self.C[:, secuencia[t]]

            # Capa oculta recurrente
            Vh = np.dot(self.V, h[t-1])
            Ux = np.dot(self.U, x_t)
            h[t] = np.tanh(Vh + Ux + self.b) # Salida de capa oculta

            # Capa de salida
            a = self.W.dot(h[t]) + self.c # Preactivación
            exp_a = np.exp(a - a.max()) # Exponencial de la preactivación
            y[t] = exp_a/exp_a.sum() # Salida Softmax
        
        return y, h
    
    def predecir(self, secuencia):
        y, h = self.forward(secuencia)
        return np.argmax(y, axis=1)

    def backprop(self, i_x, i_y, prob_salida, h_i, lr=0.1):
        y_pred = np.argmax(prob_salida) # El índice de la palabra que predijo
        
        # Backprop
        # Copio el arreglo para no modificar pesos de la salida original
        d_out = np.array(prob_salida, copy=True) 
        d_out[i_y] -= 1  # p(w_k | w_i) - y_k
        d_h = (1-h_i**2)*np.dot(d_out.T, self.U)
        d_c = np.dot(d_h.T, self.W)

        # Actualizamos los pesos
        self.U -= lr*np.outer(d_out, h_i) 
        self.c -= lr*d_out
        self.W -= lr*np.outer(d_h, self.C[:,i_x]) 
        self.b -= lr*d_h
        self.C[:, i_x] -= lr*d_c # Las demás filas no nos interesan, porque son 0
    
    def entrenar(self, epochs=50, lr=[]):
        entr_timeline = [] # Entropía a través de las épocas
        min_entr = np.inf
        for epoch in nbtqdm(range(epochs)):
            np.random.shuffle(bigramas)
            cross_entropy = 0
            for bigrama in self.bigramas:
                i_x = bigrama[0] # El índice de la primer palabra del bigrama
                i_y = bigrama[1] # El índice de la segunda palabra del bigrama
                # print(f'  Bigrama: {inv_vocab[i_x]} {inv_vocab[i_y]}')
                prob_salida, h_i = self.forward(i_x)
                # print(f'  Predicción: {inv_vocab[i_x]} {inv_vocab[np.argmax(prob_salida)]}')
                self.backprop(i_x, i_y, prob_salida, h_i, lr[epoch])
                cross_entropy -= np.log(prob_salida[i_y])
                        
            # Si la entropua actual es mejor que la menor...
            if cross_entropy < min_entr:
                min_entr = cross_entropy  # ponemos la actual
                # y movemos los mejores pesos
                self.best_C = self.C
                self.best_W = self.W
                self.best_b = self.b
                self.best_U = self.U
                self.best_c = self.c
                
            entr_timeline.append(cross_entropy)    
            tqdm.write(f'Epoch: {epoch+1} \tEntropía cruzada: {cross_entropy}')
        return entr_timeline
    
    def cargar_mejores_pesos(self):
        self.C = self.best_C
        self.W = self.best_W
        self.b = self.best_b
        self.U = self.best_U
        self.c = self.best_c
    
    def guardar_pesos(self, archivo):
        """Guarda los pesos del modelo en formato .npz
        """
        try:
            np.savez(
                archivo, 
                C = self.C, 
                W = self.W, 
                b = self.b, 
                U = self.U, 
                c = self.c
            )
            print(f'Archivo {archivo} guardado satisfactoriamente')
            return True
        except Exception as e:
            print('Ocurrió un error al guardar el archivo')
            print(e)
            return False
    
    def cargar_pesos(self, archivo):
        """Carga los pesos del modelo guardados en un archivo formato .npz
        """
        try:
            pesos = np.load(archivo)
            self.C = pesos['C']
            self.W = pesos['W']
            self.b = pesos['b']
            self.U = pesos['U']
            self.c = pesos['c']
            print(f'Pesos desde {archivo} cargados correctamente')
            return True
        except Exception as e:
            print('Ocurrió un error al guardar el archivo')
            print(e)
            return False
        

In [16]:
dim_in = len(index_to_word)
dim_out = len(tag_to_index)

dim_m = 5
dim_d = 10

In [17]:
modelo = Modelo(d = dim_d, m=dim_m, N_in = dim_in, N_out = dim_out)
modelo.inicializar_pesos()

In [18]:
# sentence = [w for w, tag in train_corpus[0]]
# y, h = modelo.forward(sentence)

In [19]:
for y_i in train_corpus[:5]:
    sentence = [w for w, tag in y_i]
    print(' '.join([ index_to_word[w] for w in sentence ]))
    pred = modelo.predecir(sentence)
    print(' '.join([ index_to_tag[tag] for tag in pred ]) )
    print()

<BOS> Officials said the total number of workers receiving unemployment benefits is now close to <unk> million , another record . <EOS>
I-per I-gpe I-gpe B-ge B-tim I-eve  I-gpe I-gpe I-per B-gpe B-gpe B-rg B-rg B-art B-ge B-gpe I-eve B-rg B-rg B-art B-ge

<BOS> The summit was dominated by the political crisis in <unk> , and the <unk> of Mr. Mwanawasa , a <unk> of <unk> President <unk> <unk> , was said to hurt attempts to put more pressure on <unk> . <EOS>
I-per B-ge I-gpe B-gpe I-ge B-rg B-rg B-ge B-gpe I-ge B-rg B-rg B-rg B-ge B-gpe I-ge B-rg B-rg B-rg B-ge B-gpe I-ge B-rg B-rg B-ge B-gpe I-eve B-rg B-rg B-ge B-gpe B-gpe I-ge B-rg B-rg B-ge B-gpe I-ge B-rg B-rg

<BOS> Earlier this year , Ethiopian Prime Minister <unk> <unk> warned an al-Qaida <unk> is already operating out of Somalia 's capital , Mogadishu . <EOS>
I-per B-rg B-rg B-rg B-rg B-ge B-ge I-eve I-ge B-rg B-rg B-ge B-ge B-gpe I-ge B-rg B-rg B-ge B-ge B-gpe  B-rg B-rg B-ge B-ge B-tim

<BOS> She did not <unk> any more details


5. Evaluar el desempeño del sistema a partir del corpus de evaluación y con la métrica de Exactitud (Accuracy).
6. Ejemplificar el reconocimiento de entidades nombradas con 5 sentencias del corpus de evaluación.